# Question 2

## 1st and 2nd pattern

In [1]:
from collections import defaultdict

VERBOSE = False 



#  Graph Definition


class Graph:
    def __init__(self, n):
        self.n = n
        self.edges = []              # list of (u, v)
        self.adj = [[] for _ in range(n)]  # for each vertex: list of (neighbor, edge_index)

    def add_edge(self, u, v):
        idx = len(self.edges)
        self.edges.append((u, v))
        self.adj[u].append((v, idx))
        self.adj[v].append((u, idx))



#  One block: pentagon + star + spokes


def add_block_edges(G, outer, inner, cycle_order=None):
    """
    outer: list of 5 outer vertex ids
    inner: list of 5 inner vertex ids
    cycle_order: permutation [0..4] describing the pentagon order
    """
    if cycle_order is None:
        cycle_order = [0, 1, 2, 3, 4]

    # Outer pentagon 
    for i in range(5):
        u = outer[cycle_order[i]]
        v = outer[cycle_order[(i + 1) % 5]]
        G.add_edge(u, v)

    # Inner star
    star_pairs = [(0, 2), (2, 4), (4, 1), (1, 3), (3, 0)]
    for a, b in star_pairs:
        G.add_edge(inner[a], inner[b])

    # Spokes
    for i in range(5):
        G.add_edge(outer[i], inner[i])



#  Build N blocks in a chain


def build_block_chain(num_blocks):
    """
    Blocks share exactly ONE outer vertex:
      Block 1: outer 0,1,2,3,4   inner 5..9
      Block 2: outer 4,10,11,12,13   inner 14..18
      Block 3: outer 13,19,20,21,22  inner 23..27
    and similarly for higher blocks.

    Each new block:
      - reuses previous block's last outer vertex
      - adds 4 new outer + 5 new inner vertices
      => total vertices = 10 + (num_blocks - 1)*9
    """
    if num_blocks <= 0:
        return Graph(0)

    total_vertices = 10 + (num_blocks - 1) * 9
    G = Graph(total_vertices)

    # -------- BLOCK 1 --------
    outer = [0, 1, 2, 3, 4]
    inner = [5, 6, 7, 8, 9]
    # For the first block we keep the natural cycle
    add_block_edges(G, outer, inner, cycle_order=[0, 1, 2, 3, 4])

    shared_vertex = 4
    next_free = 10

    # -------- BLOCKS 2..N --------
    for block in range(2, num_blocks + 1):
        # Outer vertices for this block:
        # index 0 = shared vertex, 1..4 = new ones
        outer = [shared_vertex]
        for _ in range(4):
            outer.append(next_free)
            next_free += 1

        # Inner vertices for this block: 5 fresh
        inner = []
        for _ in range(5):
            inner.append(next_free)
            next_free += 1

        add_block_edges(G, outer, inner, cycle_order=[0, 1, 2, 4, 3])

        # Last outer of this block becomes the shared vertex for next block
        shared_vertex = outer[4]

    return G



#  Lower bound for k


def compute_lower_bound(G):
    m = len(G.edges)
    delta = max(len(G.adj[u]) for u in range(G.n)) if G.n > 0 else 0
    # m <= 2k - 1  -> k >= (m+1)/2
    return max((m + 1) // 2, delta)



#  BACKTRACKING (vertex k-labeling)


def try_label_with_k(G, k):
    """
    Try to find a vertex labeling with labels in {1..k} such that
    all edge weights (sum of endpoint labels) are distinct.

    Returns (found: bool, labels: list[int])
    """

    n = G.n
    label = [0] * n
    edge_active = [False] * len(G.edges)
    used_weights = set()

    # Simple order: 0..n-1
    order = list(range(n))

    steps = 0

    def dfs(i):

        if i == n:
            return True

        u = order[i]

        for L in range(1, k + 1):
            label[u] = L

            if VERBOSE:
                print(f"Assigning Vertex {u} -> {L}")

            ok = True
            new_edges = []
            new_weights = []

            # Process all edges (u, v) that become fully labeled now
            for v, e_idx in G.adj[u]:
                if label[v] != 0 and not edge_active[e_idx]:
                    w = L + label[v]
                    if VERBOSE:
                        print(f"   Edge ({u},{v}) weight = {w}")
                    if w in used_weights:
                        if VERBOSE:
                            print(f"   ❌ Conflict weight {w}, backtrack")
                        ok = False
                        break
                    used_weights.add(w)
                    edge_active[e_idx] = True
                    new_edges.append(e_idx)
                    new_weights.append(w)

            if not ok:
                # undo partial updates from this label choice
                for e_idx, w in zip(new_edges, new_weights):
                    used_weights.remove(w)
                    edge_active[e_idx] = False
                label[u] = 0
                continue

            # Recurse
            if dfs(i + 1):
                return True

            # Backtrack
            if VERBOSE:
                print(f"↩ Backtracking vertex {u}")

            label[u] = 0
            for e_idx, w in zip(new_edges, new_weights):
                used_weights.remove(w)
                edge_active[e_idx] = False

        return False

    found = dfs(0)
    return found, label



#  FIND ES(G)


def find_edge_irregularity_strength(G):
    LB = compute_lower_bound(G)
    k = LB

    while True:
        print(f"\nTrying k = {k}")
        found, label = try_label_with_k(G, k)

        if not found:
            print("No solution, increasing k")
            k += 1
            continue

        # final safety: verify all edge weights distinct
        seen = set()
        dup = False
        for (u, v) in G.edges:
            w = label[u] + label[v]
            if w in seen:
                print("Duplicate found in final check, raising k")
                dup = True
                break
            seen.add(w)

        if not dup:
            return k, label

        k += 1



#  OUTPUT


def print_full_solution(G, labels, k):
    print("======================")
    print(" FINAL GRAPH DETAILS")
    print("======================")
    print(f"Vertices: {G.n}")
    print(f"Edges: {len(G.edges)}")
    print(f"es(G) = {k}")

    print("\n--- Vertex Labels ---")
    for v in range(G.n):
        print(f"Vertex {v}: Label = {labels[v]}")

    print("\n--- Adjacency List ---")
    for v in range(G.n):
        neighbors = [nbr for (nbr, _) in G.adj[v]]
        print(f"{v}: {neighbors}")

    print("\n--- Edges and Weights ---")
    used = set()
    for (u, v) in G.edges:
        w = labels[u] + labels[v]
        print(f"({u}, {v}) : {labels[u]} + {labels[v]} = {w}")
        if w in used:
            print("  >>> ERROR: DUPLICATE DETECTED")
        used.add(w)



#  MAIN


if __name__ == "__main__":
    NUM_BLOCKS = 5   # Num of blocks

    G = build_block_chain(NUM_BLOCKS)
    print(f"Graph with {NUM_BLOCKS} blocks | V={G.n} E={len(G.edges)}")

    k, labels = find_edge_irregularity_strength(G)
    print_full_solution(G, labels, k)

Graph with 5 blocks | V=46 E=75

Trying k = 38
 FINAL GRAPH DETAILS
Vertices: 46
Edges: 75
es(G) = 38

--- Vertex Labels ---
Vertex 0: Label = 1
Vertex 1: Label = 1
Vertex 2: Label = 2
Vertex 3: Label = 2
Vertex 4: Label = 4
Vertex 5: Label = 6
Vertex 6: Label = 7
Vertex 7: Label = 8
Vertex 8: Label = 9
Vertex 9: Label = 5
Vertex 10: Label = 13
Vertex 11: Label = 5
Vertex 12: Label = 15
Vertex 13: Label = 15
Vertex 14: Label = 17
Vertex 15: Label = 9
Vertex 16: Label = 18
Vertex 17: Label = 16
Vertex 18: Label = 19
Vertex 19: Label = 9
Vertex 20: Label = 17
Vertex 21: Label = 12
Vertex 22: Label = 24
Vertex 23: Label = 14
Vertex 24: Label = 23
Vertex 25: Label = 25
Vertex 26: Label = 26
Vertex 27: Label = 20
Vertex 28: Label = 22
Vertex 29: Label = 25
Vertex 30: Label = 24
Vertex 31: Label = 26
Vertex 32: Label = 28
Vertex 33: Label = 31
Vertex 34: Label = 29
Vertex 35: Label = 31
Vertex 36: Label = 32
Vertex 37: Label = 30
Vertex 38: Label = 35
Vertex 39: Label = 34
Vertex 40: Label =

## 3rd pattern

In [ ]:
from collections import defaultdict

VERBOSE = False   # set True if you want to see step-by-step backtracking


# Graph Definition

class Graph:
    def __init__(self, n):
        self.n = n
        self.edges = []                 # list of (u, v)
        self.adj = [[] for _ in range(n)]  # adjacency list with edge index

    def add_edge(self, u, v):
        idx = len(self.edges)
        self.edges.append((u, v))
        self.adj[u].append((v, idx))
        self.adj[v].append((u, idx))


# One block = pentagon + inner star + spokes

def add_block_edges(G, outer, inner):
    # Outer pentagon
    for i in range(5):
        G.add_edge(outer[i], outer[(i + 1) % 5])

    # Inner star
    star_pairs = [(0,2),(2,4),(4,1),(1,3),(3,0)]
    for a,b in star_pairs:
        G.add_edge(inner[a], inner[b])

    # Spokes
    for i in range(5):
        G.add_edge(outer[i], inner[i])


# STAR CHAIN BUILDER (matches your diagram)

def build_star_chain(num_blocks):
    """
    Each block shares EXACTLY ONE outer vertex.
    No bridge edges. No illegal adjacency.
    """

    if num_blocks <= 0:
        return Graph(0)

    total_vertices = 10 + (num_blocks - 1) * 9
    G = Graph(total_vertices)

    # ----- Block 1 -----
    outer = [0,1,2,3,4]
    inner = [5,6,7,8,9]
    add_block_edges(G, outer, inner)

    shared = 4
    next_free = 10

    # ----- Remaining blocks -----
    for _ in range(2, num_blocks + 1):
        outer = [shared]
        for _ in range(4):
            outer.append(next_free)
            next_free += 1

        inner = []
        for _ in range(5):
            inner.append(next_free)
            next_free += 1

        add_block_edges(G, outer, inner)
        shared = outer[4]

    return G


# Lower Bound for k

def compute_lower_bound(G):
    m = len(G.edges)
    delta = max(len(G.adj[u]) for u in range(G.n)) if G.n > 0 else 0
    return max((m + 1)//2, delta)


# Vertex k-labeling using Backtracking

def try_label_with_k(G, k, step_limit=2_000_000):

    n = G.n
    label = [0]*n
    edge_active = [False]*len(G.edges)
    used_weights = set()
    order = list(range(n))
    steps = 0

    def dfs(i):
        nonlocal steps
        steps += 1
        if steps > step_limit:
            return False

        if i == n:
            return True

        u = order[i]

        for L in range(1, k+1):
            label[u] = L
            ok = True
            new_edges = []
            new_weights = []

            for v, e_idx in G.adj[u]:
                if label[v] != 0 and not edge_active[e_idx]:
                    w = L + label[v]
                    if w in used_weights:
                        ok = False
                        break
                    used_weights.add(w)
                    edge_active[e_idx] = True
                    new_edges.append(e_idx)
                    new_weights.append(w)

            if not ok:
                for e_idx, w in zip(new_edges, new_weights):
                    used_weights.remove(w)
                    edge_active[e_idx] = False
                label[u] = 0
                continue

            if dfs(i + 1):
                return True

            label[u] = 0
            for e_idx, w in zip(new_edges, new_weights):
                used_weights.remove(w)
                edge_active[e_idx] = False

        return False

    return dfs(0), label


# Find Edge Irregularity Strength

def find_edge_irregularity_strength(G):
    k = compute_lower_bound(G)

    while True:
        print(f"Trying k = {k}")
        found, labels = try_label_with_k(G, k)

        if not found:
            k += 1
            continue

        seen = set()
        valid = True
        for u, v in G.edges:
            w = labels[u] + labels[v]
            if w in seen:
                valid = False
                break
            seen.add(w)

        if valid:
            return k, labels
        k += 1


# Output

def print_graph(G, labels, k):
    print("\nFINAL GRAPH")
    print(f"Vertices: {G.n}")
    print(f"Edges: {len(G.edges)}")
    print(f"es(G) = {k}")

    print("\nVertex Labels:")
    for i,l in enumerate(labels):
        print(f"{i}: {l}")

    print("\nAdjacency List:")
    for i in range(G.n):
        neighbors = [v for v,_ in G.adj[i]]
        print(f"{i}: {neighbors}")

    print("\nEdges and Weights:")
    for u,v in G.edges:
        print(f"({u},{v}) = {labels[u]} + {labels[v]} = {labels[u]+labels[v]}")


# MAIN

if __name__ == "__main__":
    NUM_BLOCKS = 10   # change this

    G = build_star_chain(NUM_BLOCKS)
    print(f"Graph with {NUM_BLOCKS} blocks | V={G.n} E={len(G.edges)}")

    k, labels = find_edge_irregularity_strength(G)
    print_graph(G, labels, k)

Graph with 10 blocks | V=91 E=150
Trying k = 75
Trying k = 76

====== FINAL GRAPH ======
Vertices: 91
Edges: 150
es(G) = 76

Vertex Labels:
0: 1
1: 1
2: 2
3: 2
4: 4
5: 6
6: 7
7: 8
8: 9
9: 5
10: 13
11: 5
12: 14
13: 16
14: 17
15: 9
16: 18
17: 15
18: 18
19: 9
20: 17
21: 11
22: 22
23: 15
24: 28
25: 24
26: 29
27: 21
28: 20
29: 26
30: 21
31: 29
32: 30
33: 28
34: 30
35: 33
36: 36
37: 24
38: 31
39: 27
40: 40
41: 30
42: 38
43: 41
44: 43
45: 36
46: 28
47: 47
48: 31
49: 48
50: 40
51: 54
52: 43
53: 53
54: 43
55: 37
56: 50
57: 39
58: 53
59: 46
60: 58
61: 50
62: 59
63: 53
64: 46
65: 56
66: 48
67: 60
68: 57
69: 63
70: 58
71: 64
72: 60
73: 56
74: 63
75: 59
76: 65
77: 66
78: 72
79: 66
80: 71
81: 68
82: 66
83: 73
84: 68
85: 70
86: 71
87: 76
88: 75
89: 76
90: 75

Adjacency List:
0: [1, 4, 5]
1: [0, 2, 6]
2: [1, 3, 7]
3: [2, 4, 8]
4: [3, 0, 9, 10, 13, 14]
5: [7, 8, 0]
6: [9, 8, 1]
7: [5, 9, 2]
8: [6, 5, 3]
9: [7, 6, 4]
10: [4, 11, 15]
11: [10, 12, 16]
12: [11, 13, 17]
13: [12, 4, 18, 19, 22, 23]
14: [16, 